### Movies for U, V, W, T, S, Tracers - Aids to visualize the flow

In [98]:
#KRM

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

import matplotlib as mpl
#%matplotlib inline
from math import *

import scipy.io

import scipy as spy

from netCDF4 import Dataset

import pylab as pl

import os 

import sys

import seaborn as sns

In [99]:
lib_path = os.path.abspath('../../Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 


In [100]:
CanyonGrid='/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run03/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)

CanyonState='/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run03/stateGlob.nc'
CanyonStateOut = Dataset(CanyonState)

CanyonPtracers='/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run03/ptracersGlob.nc'
CanyonPtracersOut = Dataset(CanyonPtracers)



In [101]:
#for dimobj in CanyonStateOut.variables.values():
#    print dimobj

In [102]:
# General input

nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 

z = CanyonStateOut.variables['Z']
Time = CanyonStateOut.variables['T']

xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')

hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid, 'HFacC')

hFacW = rout.getField(CanyonGrid, 'HFacW') # (Z,Y,Xp1) for zonal fluxes
MaskW = rout.getMask(CanyonGrid,'HFacW') 

hFacS = rout.getField(CanyonGrid, 'HFacS') # (Z,Yp1,X) for meridional fluxes
MaskS = rout.getMask(CanyonGrid,'HFacS') 


In [103]:
# Load velocity components
uu = rout.getField(CanyonState,'U')
vv = rout.getField(CanyonState,'V')
W = rout.getField(CanyonState,'W')

U,V = rout.unstagger(uu,vv)



In [104]:
print(np.max(uu), np.min(uu))
print(np.max(vv), np.min(vv))
print(np.max(W), np.min(W))

Umax = np.max(uu)
Wmax = np.max(W)

(0.44568375, -0.15460873)
(0.15350412, -0.16445601)
(0.0029693937, -0.0037787)


In [105]:
import matplotlib.animation as animation

In [106]:
sns.set()
sns.set_style('white')
sns.set_context("talk")
spdcmap = sns.cubehelix_palette(light=1, as_cmap=True)
divmap = sns.diverging_palette(255, 100, l=60, n=7, center="dark", as_cmap=True)

In [107]:
def Plot1(t,ax1):
    Uplot=np.ma.array(U[t,:,:,xslice],mask=MaskC[:,1:,xslice])
    csU = np.linspace(-1.0,1.0,num=26)
    csU2 = np.linspace(-1.0,1.0,num=15)
    ax1.clear()
    mesh=ax1.contourf(yc[100:-1,200],z[:58],Uplot[:58,100:]/Umax,csU,cmap='RdYlGn_r')
    if t == 1:    
        cbar=plt.colorbar(mesh,ax=ax1,cmap ="RdYlGn_r")
    CS = ax1.contour(yc[100:-1,200],z[:58],Uplot[:58,100:]/Umax,csU2,colors='k',linewidths=[0.75] )
    ax1.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax1.set_xlabel('Across-shore distance [km]')
    ax1.set_xticks([40000,50000,60000,70000,80000])
    ax1.set_xticklabels(['40', '50', '60', '70', '80'])
    ax1.set_ylabel('Depth [m]')
    ax1.set_title('Alongshore velocity at day %0.1f' %(t/2.0))

def Plot2(t,ax2):    
    Uplot=np.ma.array(U[t,:,yslice,:],mask=MaskC[:,yslice,:])
    csU = np.linspace(-0.8,0.8,num=25)
    csU2 = np.linspace(-0.8,0.8,num=13)
    ax2.clear()
    mesh=ax2.contourf(xc[200,100:260],z[:58],Uplot[:58,100:260]/Umax,csU,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax2)
    CS=ax2.contour(xc[200,100:260],z[:58],Uplot[:58,100:260]/Umax,csU2,colors='k',linewidths=[0.75] )
    ax2.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax2.set_xlabel('Alongshore distance [km]')
    ax2.set_xticks([50000,55000,60000,65000, 70000])
    ax2.set_xticklabels(['50','55','60','65','70'])
    ax2.set_ylabel('Depth [m]')
    ax2.set_title('Alongshore velocity')
    
def Plot3(t,ax3):    
    uplane = np.ma.array(U[t,zslice2,:,1:],mask=MaskC[zslice2,1:,1:])
    vplane = np.ma.array(V[t,zslice2,1:,:],mask=MaskC[zslice2,1:,1:])
    spd = np.sqrt(uplane**2+vplane**2)
    csSpd = np.linspace(0,1,num=25)
    ax3.clear()
    mesh=ax3.contourf(xc[145:265,1:],yc[145:265,1:],spd[145:265,:]/Umax,csSpd,cmap=spdcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax3)
    CS = ax3.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[145],colors='k',linewidths=[2.0] )
    vect = ax3.quiver(xc[145:265:5, 1::20], yc[145:265:5, 1::20], uplane[145:265:5, ::20]/Umax, vplane[145:265:5, ::20]/Umax
                      , color='k')
    
    MerPlane = ax3.plot(xc[yslice,:],yc[yslice,:],'k:',linewidth = 2)
    ZonPlane = ax3.plot(xc[145:265,xslice],yc[145:265,xslice],'k:',linewidth = 2)
    ax3.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax3.set_xlabel('Alongshore distance [km]')
    ax3.set_xticks([20000,40000,60000,80000,100000])
    ax3.set_xticklabels(['20', '40', '60', '80', '100'])
    ax3.set_ylabel('Across-shore distance [km]')
    ax3.set_yticks([40000,45000,50000,55000])
    ax3.set_yticklabels(['40', '45', '50', '55'])
    ax3.set_title('Horizontal velocity above shelf break')

def Plot4(t,ax4):
    Vplot=np.ma.array(V[t,:,:,xslice],mask=MaskC[:,:,xslice])
    csV = np.linspace(-0.22,0.22,num=25)
    csV2 = np.linspace(-0.22,0.22,num=16)
    ax4.clear()
    mesh=ax4.contourf(yc[100:,200],z[:58],Vplot[:58,100:]/Umax,csV,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax4)
    CS = ax4.contour(yc[100:,200],z[:58],Vplot[:58,100:]/Umax,csV2,colors='k',linewidths=[0.75] )
    ax4.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax4.set_xlabel('Across-shore distance [km]')
    ax4.set_xticks([40000,50000,60000,70000,80000])
    ax4.set_xticklabels(['40', '50', '60', '70', '80'])
    ax4.set_ylabel('Depth [m]')
    ax4.set_title('Across-shore velocity')
    
def Plot5(t,ax5):    
    Vplot=np.ma.array(V[t,:,yslice,:],mask=MaskC[:,yslice,1:])
    csV = np.linspace(-0.32,0.32,num=25)
    csV2 = np.linspace(-0.32,0.32,num=13)
    ax5.clear()
    mesh=ax5.contourf(xc[200,100:260],z[:58],Vplot[:58,100:260]/Umax,csV,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax5)
    CS=ax5.contour(xc[200,100:260],z[:58],Vplot[:58,100:260]/Umax,csV2,colors='k',linewidths=[0.75] )
    ax5.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax5.set_xlabel('Alongshore distance [km]')
    ax5.set_xticks([50000,55000,60000,65000, 70000])
    ax5.set_xticklabels(['50','55','60','65','70'])
    ax5.set_ylabel('Depth [m]')
    ax5.set_title('Across-shore velocity')

def Plot6(t,ax6):
    uplane = np.ma.array(U[t,zslice,:,1:],mask=MaskC[zslice,1:,1:])
    vplane = np.ma.array(V[t,zslice,1:,:],mask=MaskC[zslice,1:,1:])
    spd = np.sqrt(uplane**2+vplane**2)
    csSpd = np.linspace(0,1,num=25)
    ax6.clear()
    mesh=ax6.contourf(xc[145:265,1:],yc[145:265,1:],spd[145:265,:]/Umax,csSpd,cmap=spdcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax6)
    vect = ax6.quiver(xc[145:265:5, 1::15], yc[145:265:5, 1::15], uplane[145:265:5, ::15]/Umax, vplane[145:265:5, ::15]/Umax,
            pivot='mid', color='k')
    CS = ax6.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[148],colors='k',linewidths=[2.25] )
    ax6.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax6.set_xlabel('Alongshore distance [km]')
    ax6.set_xticks( [20000,40000,60000,80000,100000])
    ax6.set_xticklabels(['20', '40', '60', '80', '100'] )
    ax6.set_ylabel('Across-shore distance [km]')
    ax6.set_yticks([40000,45000,50000,55000])
    ax6.set_yticklabels(['40', '45', '50', '55'])
    ax6.set_title('Horizontal velocity at shelf break')

def Plot7(t,ax7):
    Wplot=np.ma.array(W[t,:,:,xslice],mask=MaskC[:,:,xslice])
    csW = np.linspace(-0.4,0.4,num=30)
    csW2 = np.linspace(-0.4,0.4,num=15)
    ax7.clear()
    mesh=ax7.contourf(yc[100:,200],z[:58],Wplot[:58,100:]/Wmax,csW,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax7)
    CS = ax7.contour(yc[100:,200],z[:58],Wplot[:58,100:]/Wmax,csW2,colors='k',linewidths=[0.75] )
    ax7.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax7.set_xlabel('Across-shore distance [km]')
    ax7.set_xticks([40000,50000,60000,70000,80000])
    ax7.set_xticklabels(['40', '50', '60', '70', '80'])
    ax7.set_ylabel('Depth [m]')
    ax7.set_title('Vertical velocity')

def Plot8(t,ax8):
    Wplot=np.ma.array(W[t,:,yslice,:],mask=MaskC[:,yslice,:])
    csW = np.linspace(-0.65,0.65,num=30)
    csW2 = np.linspace(-0.65,0.65,num=15)
    ax8.clear()
    mesh=ax8.contourf(xc[200,100:260],z[:58],Wplot[:58,100:260]/Wmax,csW,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax8)
    CS=ax8.contour(xc[200,100:260],z[:58],Wplot[:58,100:260]/Wmax,csW2,colors='k',linewidths=[0.75])
    ax8.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax8.set_xlabel('Alongshore distance [km]')
    ax8.set_xticks([50000,55000,60000,65000, 70000])
    ax8.set_xticklabels(['50','55','60','65','70'])
    ax8.set_ylabel('Depth [m]')
    ax8.set_title('Vertical velocity')
    
def Plot9(t,ax9):
    wplane = np.ma.array(W[t,zslice,:,:],mask=MaskC[zslice,:,:])
    csW = np.linspace(-1,1,num=30)
    csW2 = np.linspace(-1,1,num=15)
    ax9.clear()
    mesh=ax9.contourf(xc[145:265,:],yc[145:265,:],wplane[145:265,:]/Wmax,csW,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax9)
    CS = ax9.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[145],colors='k',linewidths=[2.25] )
    MerPlane = ax9.plot(xc[yslice,:],yc[yslice,:],'k:',linewidth = 2.0)
    ZonPlane = ax9.plot(xc[145:265,xslice],yc[145:265,xslice],'k:',linewidth = 2.0)
    CS=ax9.contour(xc[145:265,:],yc[145:265,:],wplane[145:265,:]/Wmax,csW2,colors='k',linewidths=[0.75])
    ax9.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    
    ax9.set_xlabel('Alongshore distance [km]')
    plt.xticks( (20000,40000,60000,80000,100000), ('20', '40', '60', '80', '100') )
    ax9.set_ylabel('Across-shore distance [km]')
    plt.yticks( (40000,45000,50000,55000), ('40', '45', '50', '55') )
    ax9.set_title('Vertical velocity at shelf break')

In [108]:
## Animation
#N=5
xslice=180
yslice=235
zslice= 29 # shelf break index
zslice2= 23 

#Empty figures
fig,((ax1,ax2,ax3),(ax4, ax5,ax6),(ax7,ax8,ax9))  = plt.subplots(3, 3, figsize=(18, 12))

#Initial image
def init():
    Plot1(0,ax1)
    Plot2(0,ax2)
    Plot3(0,ax3)
    Plot4(0,ax4)
    Plot5(0,ax5)
    Plot6(0,ax6)
    Plot7(0,ax7)
    Plot8(0,ax8)
    Plot9(0,ax9)
    
    plt.tight_layout()
    #return[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]
    

def animate(tt):
    Plot1(tt,ax1)
    Plot2(tt,ax2)
    Plot3(tt,ax3)
    Plot4(tt,ax4)
    Plot5(tt,ax5)
    Plot6(tt,ax6)
    Plot7(tt,ax7)
    Plot8(tt,ax8)
    Plot9(tt,ax9)
    plt.tight_layout()
#The animation function (max frames=47)   


anim = animation.FuncAnimation(fig, animate, init_func=init,frames=18, interval = 200,blit=False, repeat=False)

##A line that makes it all work
mywriter = animation.FFMpegWriter()

##Save in current folder
anim.save('Vel_3DDIFF_run03.mp4',writer=mywriter,fps=0.2, dpi = 300, bitrate = 1000000, codec = "libx264")

plt.show()

